In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import numpy as np
import pandas as pd
from baynes import FitPlotter, get_model, hdi

# Model comparison
### Compare simple linear fits with or without intercept

In [ ]:
N = 21
x = np.linspace(0, 10, N)

alpha = 0.5
beta = 0.4
sigma = 0.5

y = alpha + beta * (x + np.random.normal(0, sigma, N))
data = {'N': N, 'x': x, 'y': y, 'prior': 0}

fplot = FitPlotter()
fplot.plot(x, y)

In [ ]:
model = get_model('linear')
model_noalpha = get_model('linear_no_intercept.stan')

In [ ]:
fit = model.sample(data)
fplot.add_fit(fit, 'with intercept')
fit_noalpha = model_noalpha.sample(data)
fplot.add_fit(fit_noalpha, 'no intercept')

## Compare posterior predictive checks

In [ ]:
ax = fplot.new_figure('linear_compare').subplots()
df_tot = pd.DataFrame()
for title, f in fplot.fits.items():
    df = f.draws_pd('y_rep').transpose()
    df['x']=x
    df['fit'] = title
    df_tot = pd.concat([df, df_tot])

ax.plot(x, y, label='data', color='black')
ax = sns.lineplot(df_tot.melt(id_vars=['x', 'fit']), x='x', y='value', hue='fit', errorbar=hdi, ax=ax, err_kws={'alpha':1}, alpha=1, zorder=0)
plt.legend(facecolor='white', edgecolor='white')
ax.set_ylabel('y')

## WAIC

In [ ]:
df = az.compare({'no intercept': fit_noalpha, 'with intercept':fit}, ic='waic')
print(df[['elpd_waic', 'se', 'p_waic']].to_latex(float_format="%.2f"))

## PSIS-LOO

In [ ]:
ax = fplot.new_figure('loo_khat').subplots()
loo_noa = az.loo(fit_noalpha)
ax = az.plot_khat(loo_noa, label = 'no intercept', ax=ax, markersize=200)
loo = az.loo(fit)
az.plot_khat(loo, label = 'with intercept', ax=ax, markersize=100, marker='o')
plt.legend(bbox_to_anchor=(1.05, 0.5), facecolor='white', edgecolor='white')

In [ ]:
df = az.compare({'no intercept': fit_noalpha, 'with intercept':fit})
print(df[['elpd_loo', 'se', 'p_loo']].to_latex(float_format="%.2f"))